**Installing Wandb**

In [ ]:
!pip install wandb

**Import statements**

In [ ]:
from keras.datasets import fashion_mnist
import wandb
import numpy as np

# **Question 1**

In [ ]:
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
k = len(class_names)
wandb.init(
    project="Assignment 1",
)
def plotImagesOfEachClass():
  image_labels = []
  images = []
  for i in range(len(trainX)):
    if len(image_labels) == len(class_names):
      break
    if class_names[trainy[i]] not in image_labels:
      image_labels.append(class_names[trainy[i]])
      images.append(trainX[i])

  wandb.log({"examples ": [wandb.Image(img, caption=caption) for img, caption in zip(images, image_labels)]})

plotImagesOfEachClass()

# **Question 2**

In [ ]:
def initializeWeightAndBias(num_hidden_layers, neurons_in_each_layer, num_images, k):
  W = {}
  bias = {}
  W["1"] = (np.random.randn(neurons_in_each_layer, 784))
  bias["1"] = (np.random.randn(neurons_in_each_layer))
  for i in range(num_hidden_layers-1):
    W[str(i+2)] = (np.random.randn(neurons_in_each_layer, neurons_in_each_layer))
    bias[str(i+2)] = (np.random.randn(neurons_in_each_layer))
  W[str(num_hidden_layers+1)] = (np.random.randn(k, neurons_in_each_layer))
  bias[str(num_hidden_layers+1)] = (np.random.randn(k))
  return W, bias

def flattenList(X):
  return X.flatten()

def sigmoid(X):
  return 1.0/(1+np.exp(-X))

def softmax(a):
  return np.exp(a) / np.sum(np.exp(a))

def feedForward(W, bias, X, num_hidden_layers, neurons_in_each_layer):
  preactivation = {}
  activation = {}
  h0 = flattenList(X)
  activation["h0"] = h0
  for i in range(1, num_hidden_layers+1):
    # print(i)
    # print(bias[str(i)].shape)
    # print(W[str(i)].shape)
    # print(activation["h"+str(i-1)].shape)
    ai = bias[str(i)] + np.matmul(W[str(i)], activation["h"+str(i-1)])
    preactivation["a"+str(i)] = ai
    hi = sigmoid(ai)
    activation["h"+str(i)] = hi
  preactivation["a"+str(num_hidden_layers+1)] = bias[str(num_hidden_layers+1)] + np.matmul(W[str(num_hidden_layers+1)], activation["h"+str(num_hidden_layers)])
  # print(preactivation["a"+str(num_hidden_layers+1)])
  activation["h"+str(num_hidden_layers+1)] = softmax(preactivation["a"+str(num_hidden_layers+1)])
  # print(activation)
  return activation["h"+str(num_hidden_layers+1)]

  






In [ ]:
num_hidden_layers = 4
neurons_in_each_layer = 32
num_images = len(trainX)
W, bias = initializeWeightAndBias(num_hidden_layers, neurons_in_each_layer, num_images, k)
probabilityDistribution = []
for i in range(num_images):
  probabilityDistribution.append(feedForward(W, bias, trainX[i], num_hidden_layers, neurons_in_each_layer))
  print(np.argmax(probabilityDistribution[i]))